<a href="https://colab.research.google.com/github/EvelyParanhos/estudo-IA-simbolica/blob/master/IA_simbolica_Backward_Chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estrutura IA Simbólica
*   **Fatos:** O conhecimento inicial.
*   **Regras:** O conhecimento (a lógica).
*   **Motor de Inferência:** O algoritmo que aplica as regras (o core da IA).	Uma função ou classe principal.

In [6]:
class Regra:
  def __init__(self, nome, condicoes_and=None, condicoes_or=None, conclusao=None):
    self.nome = nome
    self.condicoes_and = condicoes_and
    self.condicoes_or = condicoes_or
    self.conclusao = conclusao

  def __str__(self):
    return f"Rega {self.nome}: -> Conclusão: {self.conclusao}"

In [7]:
regras_do_sistema = [
    Regra(
        nome="Regra_Gripe",
        condicoes_and=["tem_febre", "está_apático"],
        condicoes_or=["tem_tosse", "tem_espirro"],
        conclusao="Diagnóstico: Resfriado/Gripe"
    ),
    Regra(
        nome="Regra_Pulga",
        condicoes_and=["tem_coceira"],
        condicoes_or=["tem_manchas_vermelhas", "perdeu_pelo_localizado"],
        conclusao="Diagnóstico: Pulgas/Alergia"
    ),
    Regra(
        nome="Regra_Fome_Ou_Tédio",
        condicoes_and=["está_latindo_muito"],
        condicoes_or=["não_comeu_recentemente", "ficou_sozinho_hoje"],
        conclusao="Diagnóstico: Fome ou Tédio"
    )
]

In [8]:
class BaseDeFatos:
  def __init__(self, fatos_iniciais=None):
    self.fatos = fatos_iniciais if fatos_iniciais is not None else {}

  def adicionar_fato(self, condicao, valor):
    self.fatos[condicao] = valor

  def checar_fato(self, condicao):
    return self.fatos.get(condicao, False)

In [19]:
class MotorDeInferencia:
  def __init__(self, regras, base_de_fatos):
    self.regras = regras
    self.base_de_fatos = base_de_fatos

  def adquirir_fato(self, condicao):
    if condicao in self.base_de_fatos.fatos:
      print(f"O fato '{condicao}' já está na base de fatos.")
      return self.base_de_fatos.checar_fato(condicao)

    print(f"Precisamos provar a condição: '{condicao}'")
    pergunta = f"O pet {condicao.replace('_', ' ')}? (s/n): "
    resposta = input(pergunta).lower() == 's'
    self.base_de_fatos.adicionar_fato(condicao, resposta)

    if resposta:
      print(f"O fato '{condicao}' foi confirmado.")
    else:
      print(f"O fato '{condicao}' não foi confirmado.")

    return resposta

  def provar_objetivo(self, objetivo):
    regra_alvo = next((r for r in self.regras if r.conclusao == objetivo), None)
    if not regra_alvo:
      print(f"Objetivo não encontrado: {objetivo}")
      return None

    print(f"\n--- TENTATIVA DE PROVA: {objetivo} (Regra: {regra_alvo.nome}) ---")

    for condicao_and in regra_alvo.condicoes_and:
      if self.base_de_fatos.checar_fato(condicao_and):
        print(f"Fato conhecido: {condicao_and}")
        continue

      if not self.adquirir_fato(condicao_and):
        print(f"Condição AND'{condicao_and}' não confirmada. Não é possível provar.")
        return None

    condicoes_or_provadas = False
    print(f"Condições AND provadas. Verificando condições OR...")

    if regra_alvo.condicoes_or:
      for condicao_or in regra_alvo.condicoes_or:
        if self.base_de_fatos.checar_fato(condicao_or):
          condicoes_or_provadas = True
          print(f"Fato conhecido: {condicao_or}")
          break

        if self.adquirir_fato(condicao_or):
          condicoes_or_provadas = True
          print(f"Condição OR '{condicao_or}' confirmada.")
          break

      if not condicoes_or_provadas:
        print(f"--- FALHA na Regra {regra_alvo.nome}: TODAS as condições OR falharam. ---")
        return None

    print(f"\n**SUCESSO**: Todas as condições (AND e OR) provadas. {objetivo}")
    return objetivo

In [ ]:
fatos_iniciais = {
    "tem_febre": True,
    "está_latindo_muito": True
}
base_fatos = BaseDeFatos(fatos_iniciais)

motor = MotorDeInferencia(regras_do_sistema, base_fatos)

objetivo_principal = "Diagnóstico: Resfriado/Gripe"
resultado = motor.provar_objetivo(objetivo_principal)

print("\n-------------------------------------")
if resultado:
    print(f"Resultado final do diagnóstico: {resultado}")
else:
    print(f"Resultado final do diagnóstico: Não foi possível provar o objetivo: {objetivo_principal}")
print("-------------------------------------")